In [5]:
import kaggle
import os
import zipfile

In [3]:
data_dir = "./Data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

Using here an initial dataset found at https://www.kaggle.com/datasets/paulchambaz/google-street-view/data

This will be replaced in the future

In [4]:
!kaggle datasets download -d paulchambaz/google-street-view -p {data_dir}

 99%|███████████████████████████████████████▊| 596M/600M [00:16<00:00, 44.9MB/s]
100%|████████████████████████████████████████| 600M/600M [00:16<00:00, 39.1MB/s]


In [6]:
with zipfile.ZipFile(f"{data_dir}/google-street-view.zip", 'r') as zip_ref:
    zip_ref.extractall(data_dir)